## 1시간 간격으로 예측모델을 만들어서 예측해 봄

# 원래 모델 > 퀀타일 9개 * 내일 & 모레 = 18개

# 시도한 모델
# 15개(6시 모델, 6시 모델, 7시 모델.... 20시 모델)
# 퀀타일 9개, 요일 2개
# 15 * 9 * 2 = 270개가 돌아가면서 submission을 채우는 방식


### 소감
### traindata가 1시간이니까, 1/24로 줄어서 각각의 모델 학습시간이 줄어들 것이라 예상했는데
# 실제로 돌려보니까 더럽게 오래걸림
# 5시간 째 돌리는 중, 예상시간 9~11시간?? 그닥 큰 실효성은 없을 듯.

In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20755 (delta 2), reused 2 (delta 0), pack-reused 20737
Receiving objects: 100% (20755/20755), 16.57 MiB | 8.81 MiB/s, done.
Resolving deltas: 100% (15146/15146), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=688d19dc6e1d94f7600d4a6375ad79bc60b27c9f22e6ce0a1d6e8710aab7e4ea
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [4]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [5]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [6]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [7]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [8]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [9]:
def train_to_supervised(train, times, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    cols.append(times)
    names += ["Time"]
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [10]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, times, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    cols.append(times)
    names += ["Time"]  
     
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [11]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [12]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [13]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [14]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [15]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [16]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [17]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [18]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [19]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [20]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset6 = dataset5[['RH', 'T', 'SH', 'GHI', 'TARGET']]
timeset = dataset5[['Time']]
dataset6.head(10)

,RH,T,SH,GHI,TARGET
0,69.08,-12,9.0,0.0,0.0
1,69.06,-12,9.0,0.0,0.0
2,71.78,-12,9.0,0.0,0.0
3,71.75,-12,9.0,0.0,0.0
4,75.20,-12,9.0,0.0,0.0
5,69.29,-11,9.0,0.0,0.0
6,72.56,-11,9.0,0.0,0.0
7,72.55,-11,9.0,0.0,0.0
8,74.62,-11,9.0,0.0,0.0
9,74.61,-11,9.0,0.0,0.0


In [21]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset6)
temp_X = pd.DataFrame(scaler.transform(dataset6), columns = dataset6.columns)
temp_y = pd.DataFrame(dataset6['TARGET'])

In [22]:
timeset.columns

Index(['Time'], dtype='object')

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [23]:
dataset7 = train_to_supervised(temp_X,timeset, temp_y, n_days)
dataset7

,RH(t-144),T(t-144),SH(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),GHI(t-139),TARGET(t-139),RH(t-138),T(t-138),SH(t-138),GHI(t-138),TARGET(t-138),RH(t-137),T(t-137),SH(t-137),GHI(t-137),TARGET(t-137),...,GHI(t-8),TARGET(t-8),RH(t-7),T(t-7),SH(t-7),GHI(t-7),TARGET(t-7),RH(t-6),T(t-6),SH(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),GHI(t-1),TARGET(t-1),Time,TARGET1(t+48),TARGET1(t+96)
144,0.665404,0.129630,0.083333,0.0,0.0,0.665188,0.129630,0.083333,0.0,0.0,0.694622,0.129630,0.083333,0.0,0.0,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,...,0.0,0.0,0.857267,0.129630,0.083333,0.0,0.0,0.814955,0.129630,0.083333,0.0,0.0,0.814847,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.0,0.0,0.0
145,0.665188,0.129630,0.083333,0.0,0.0,0.694622,0.129630,0.083333,0.0,0.0,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,...,0.0,0.0,0.814955,0.129630,0.083333,0.0,0.0,0.814847,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.5,0.0,0.0
146,0.694622,0.129630,0.083333,0.0,0.0,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,0.725246,0.148148,0.083333,0.0,0.0,...,0.0,0.0,0.814847,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.759442,0.111111,0.083333,0.0,0.0,1.0,0.0,0.0
147,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,0.725246,0.148148,0.083333,0.0,0.0,0.715832,0.148148,0.083333,0.0,0.0,...,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.759442,0.111111,0.083333,0.0,0.0,0.758576,0.111111,0.083333,0.0,0.0,1.5,0.0,0.0
148,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,0.725246,0.148148,0.083333,0.0,0.0,0.715832,0.148148,0.083333,0.0,0.0,0.715723,0.148148,0.083333,0.0,0.0,...,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.759442,0.111111,0.083333,0.0,0.0,0.758576,0.111111,0.083333,0.0,0.0,0.692349,0.111111,0.083333,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,0.836057,0.203704,0.083333,0.0,0.0,0.808029,0.203704,0.083333,0.0,0.0,0.807921,0.203704,0.083333,0.0,0

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [24]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
# file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [25]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [26]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [27]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [28]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [29]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [30]:
testset6 = testset5[['RH', 'T', 'SH', 'GHI', 'TARGET']]
timeset = testset5[['Time']]
testset6.head(10)

,RH,T,SH,GHI,TARGET
0,70.34,-6.1,9.5,0.0,0.0
1,71.97,-6.4,9.5,0.0,0.0
2,69.77,-6.7,9.5,0.0,0.0
3,71.95,-7.1,9.5,0.0,0.0
4,69.48,-7.4,9.5,0.0,0.0
5,72.13,-7.9,9.5,0.0,0.0
6,69.76,-8.3,9.5,0.0,0.0
7,71.97,-8.7,9.5,0.0,0.0
8,68.56,-9.1,9.5,0.0,0.0
9,70.75,-9.5,9.5,0.0,0.0


In [31]:
testset6 = pd.DataFrame(scaler.transform(testset6), columns = testset6.columns)

In [32]:
testset7 = test_to_supervised(testset6, timeset, n_days)
testset7

,RH(t-144),T(t-144),SH(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),GHI(t-139),TARGET(t-139),RH(t-138),T(t-138),SH(t-138),GHI(t-138),TARGET(t-138),RH(t-137),T(t-137),SH(t-137),GHI(t-137),TARGET(t-137),...,T(t-8),SH(t-8),GHI(t-8),TARGET(t-8),RH(t-7),T(t-7),SH(t-7),GHI(t-7),TARGET(t-7),RH(t-6),T(t-6),SH(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),GHI(t-1),TARGET(t-1),Time
144,0.679039,0.238889,0.166667,0.0,0.0,0.696678,0.233333,0.166667,0.0,0.0,0.672871,0.227778,0.166667,0.0,0.0,0.696461,0.220370,0.166667,0.0,0.0,0.669733,0.214815,0.166667,0.0,0.0,0.698409,0.205556,0.166667,0.0,0.0,0.672763,0.198148,0.166667,0.0,0.0,0.696678,0.190741,0.166667,0.0,0.0,...,0.266667,0.166667,0.0,0.0,0.436857,0.266667,0.166667,0.0,0.0,0.458284,0.266667,0.166667,0.0,0.0,0.454172,0.268519,0.166667,0.0,0.0,0.481333,0.268519,0.166667,0.0,0.0,0.480684,0.268519,0.166667,0.0,0.0,0.499188,0.270370,0.166667,0.0,0.0,0.494752,0.272222,0.166667,0.0,0.0,0.0
145,0.696678,0.233333,0.166667,0.0,0.0,0.672871,0.227778,0.166667,0.0,0.0,0.696461,0.220370,0.166667,0.0,0.0,0.669733,0.214815,0.166667,0.0,0.0,0.698409,0.205556,0.166667,0.0,0.0,0.672763,0.198148,0.166667,0.0,0.0,0.696678,0.190741,0.166667,0.0,0.0,0.659777,0.183333,0.166667,0.0,0.0,...,0.266667,0.166667,0.0,0.0,0.458284,0.266667,0.166667,0.0,0.0,0.454172,0.268519,0.166667,0.0,0.0,0.481333,0.268519,0.166667,0.0,0.0,0.480684,0.268519,0.166667,0.0,0.0,0.499188,0.270370,0.166667,0.0,0.0,0.494752,0.272222,0.166667,0.0,0.0,0.512282,0.274074,0.166667,0.0,0.0,0.5
146,0.672871,0.227778,0.166667,0.0,0.0,0.696461,0.220370,0.166667,0.0,0.0,0.669733,0.214815,0.166667,0.0,0.0,0.698409,0.205556,0.166667,0.0,0.0,0.672763,0.198148,0.166667,0.0,0.0,0.696678,0.190741,0.166667,0.0,0.0,0.659777,0.183333,0.166667,0.0,0.0,0.683476,0.175926,0.166667,0.0,0.0,...,0.266667,0.166667,0.0,0.0,0.454172,0.268519,0.166667,0.0,0.0,0.481333,0.268519,0.166667,0.0,0.0,0.480684,0.268519,0.166667,0.0,0.0,0.499188,0.270370,0.166667,0.0,0.0,0.494752,0.272222,0.166667,0.0,0.0,0.512282,0.274074,0.166667,0.0,0.0,0.507845,0.275926,0.166667,0.0,0.0,1.0
147,0.696461,0.220370,0.166667,0.0,0.0,0.669733,0.214815,0.166667,0.0,0.0,0.698409,0.205556,0.166667,0.0,0.0,0.672763,0.198148,0.166667,0.0,0.0,0.696678,0.190741,0.166667,0.0,0.0,0.659777,0.183333,0.166667,0.0,0.0,0.683476,0.175926,0.166667,0.0,0.0,0.637485,0.170370,0.166667,0.0,0.0,...,0.268519,0.166667,0.0,0.0,0.481333,0.268519,0.166667,0.0,0.0,0.480684,0.268519,0.166667,0.0,0.0,0.499188,0.270370,0.166667,0.0,0.0,0.494752,0.272222,0.166667,0.0,0.0,0.512282,0.274074,0.166667,0.0,0.0,0.507845,0.275926,0.166667,0.0,0.0,0.532626,0.277778,0.166667,0.0,0.0,1.5
148,0.669733,0.214815,0.166667,0.0,0.0,0.698409,0.205556,0.166667,0.0,0.0,0.672763,0.198148,0.166667,0.0,0.0,0.696678,0.190741,0.166667,0.0,0.0,0.659777,0.183333,0.166667,0.0,0.0,0.683476,0.175926,0.166667,0.0,0.0,0.637485,0.170370,0.166667,0.0,0.0,0.660535,0.162963,0.166667,0.0,0.0,...,0.268519,0.166667,0.0,0.0,0.480684,0.268519,0.166667,0.0,0.0,0.499188,0.270370,0.166667,0.0,0.0,0.494752,0.272222,0.166667,0.0,0.0,0.512282,0.274074,0.166667,0.0,0.0,0.507845,0.275926,0.166667,0.0,0.0,0.532626,0.277778,0.166667,0.0,0.0,0.522779,0.281481,0.166667,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.575695,0.287037,0.166667,0.0,0.0,0.570393,0.281481,0.166667,0.0,0

In [33]:
testset9 = testset7.iloc[-48:,:] #season을 다시 붙인다면 testset7->8 근데 여기 이해가 안돼.. 48 뭐임? 하루치?

In [34]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['RH', 'T', 'SH', 'GHI', 'TARGET']] #'DHI', 'DNI', 'season', 'WS', 'SH'
    timeset = testset5[['Time']]
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, timeset, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),T(t-144),SH(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),GHI(t-139),TARGET(t-139),RH(t-138),T(t-138),SH(t-138),GHI(t-138),TARGET(t-138),RH(t-137),T(t-137),SH(t-137),GHI(t-137),TARGET(t-137),...,T(t-8),SH(t-8),GHI(t-8),TARGET(t-8),RH(t-7),T(t-7),SH(t-7),GHI(t-7),TARGET(t-7),RH(t-6),T(t-6),SH(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),GHI(t-1),TARGET(t-1),Time
288,0.557299,0.353704,0.166667,0.0,0.0,0.580132,0.344444,0.166667,0.0,0.0,0.629477,0.337037,0.166667,0.0,0.0,0.634672,0.335185,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.655557,0.333333,0.166667,0.0,0.0,0.650254,0.335185,0.166667,0.0,0.0,...,0.307407,0.166667,0.000000,0.000000,0.823504,0.305556,0.166667,0.000000,0.000000,0.813224,0.303704,0.166667,0.000000,0.000000,0.813224,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.0,0.0,0.798507,0.303704,0.166667,0.0,0.0,0.805108,0.301852,0.166667,0.0,0.0,0.0
289,0.580132,0.344444,0.166667,0.0,0.0,0.629477,0.337037,0.166667,0.0,0.0,0.634672,0.335185,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.655557,0.333333,0.166667,0.0,0.0,0.650254,0.335185,0.166667,0.0,0.0,0.613678,0.338889,0.166667,0.0,0.0,...,0.305556,0.166667,0.000000,0.000000,0.813224,0.303704,0.166667,0.000000,0.000000,0.813224,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.798507,0.303704,0.166667,0.0,0.0,0.805108,0.301852,0.166667,0.0,0.0,0.793529,0.300000,0.166667,0.0,0.0,0.5
290,0.629477,0.337037,0.166667,0.0,0.0,0.634672,0.335185,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.655557,0.333333,0.166667,0.0,0.0,0.650254,0.335185,0.166667,0.0,0.0,0.613678,0.338889,0.166667,0.0,0.0,0.609458,0.340741,0.166667,0.0,0.0,...,0.303704,0.166667,0.000000,0.000000,0.813224,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.798507,0.303704,0.166667,0.000000,0.000000,0.805108,0.301852,0.166667,0.0,0.0,0.793529,0.300000,0.166667,0.0,0.0,0.800130,0.298148,0.166667,0.0,0.0,1.0
291,0.634672,0.335185,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.655557,0.333333,0.166667,0.0,0.0,0.650254,0.335185,0.166667,0.0,0.0,0.613678,0.338889,0.166667,0.0,0.0,0.609458,0.340741,0.166667,0.0,0.0,0.581647,0.342593,0.166667,0.0,0.0,...,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.798507,0.303704,0.166667,0.000000,0.000000,0.805108,0.301852,0.166667,0.000000,0.000000,0.793529,0.300000,0.166667,0.0,0.0,0.800130,0.298148,0.166667,0.0,0.0,0.782599,0.296296,0.166667,0.0,0.0,1.5
292,0.672113,0.333333,0.166667,0.0,0.0,0.672113,0.333333,0.166667,0.0,0.0,0.655557,0.333333,0.166667,0.0,0.0,0.650254,0.335185,0.166667,0.0,0.0,0.613678,0.338889,0.166667,0.0,0.0,0.609458,0.340741,0.166667,0.0,0.0,0.581647,0.342593,0.166667,0.0,0.0,0.576886,0.344444,0.166667,0.0,0.0,...,0.303704,0.166667,0.000000,0.000000,0.805216,0.303704,0.166667,0.000000,0.000000,0.798507,0.303704,0.166667,0.000000,0.000000,0.805108,0.301852,0.166667,0.000000,0.000000,0.793529,0.300000,0.166667,0.000000,0.000000,0.800130,0.298148,0.166667,0.0,0.0,0.782599,0.296296,0.166667,0.0,0.0,0.782599,0.296296,0.166667,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [35]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
X

,RH(t-144),T(t-144),SH(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),GHI(t-139),TARGET(t-139),RH(t-138),T(t-138),SH(t-138),GHI(t-138),TARGET(t-138),RH(t-137),T(t-137),SH(t-137),GHI(t-137),TARGET(t-137),...,T(t-8),SH(t-8),GHI(t-8),TARGET(t-8),RH(t-7),T(t-7),SH(t-7),GHI(t-7),TARGET(t-7),RH(t-6),T(t-6),SH(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),GHI(t-1),TARGET(t-1),Time
144,0.665404,0.129630,0.083333,0.0,0.0,0.665188,0.129630,0.083333,0.0,0.0,0.694622,0.129630,0.083333,0.0,0.0,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,...,0.129630,0.083333,0.0,0.0,0.857267,0.129630,0.083333,0.0,0.0,0.814955,0.129630,0.083333,0.0,0.0,0.814847,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.0
145,0.665188,0.129630,0.083333,0.0,0.0,0.694622,0.129630,0.083333,0.0,0.0,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,...,0.129630,0.083333,0.0,0.0,0.814955,0.129630,0.083333,0.0,0.0,0.814847,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.5
146,0.694622,0.129630,0.083333,0.0,0.0,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,0.725246,0.148148,0.083333,0.0,0.0,...,0.129630,0.083333,0.0,0.0,0.814847,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.759442,0.111111,0.083333,0.0,0.0,1.0
147,0.694297,0.129630,0.083333,0.0,0.0,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,0.725246,0.148148,0.083333,0.0,0.0,0.715832,0.148148,0.083333,0.0,0.0,...,0.129630,0.083333,0.0,0.0,0.773726,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.759442,0.111111,0.083333,0.0,0.0,0.758576,0.111111,0.083333,0.0,0.0,1.5
148,0.731631,0.129630,0.083333,0.0,0.0,0.667677,0.148148,0.083333,0.0,0.0,0.703062,0.148148,0.083333,0.0,0.0,0.702954,0.148148,0.083333,0.0,0.0,0.725354,0.148148,0.083333,0.0,0.0,0.725246,0.148148,0.083333,0.0,0.0,0.715832,0.148148,0.083333,0.0,0.0,0.715723,0.148148,0.083333,0.0,0.0,...,0.111111,0.083333,0.0,0.0,0.773618,0.111111,0.083333,0.0,0.0,0.795368,0.111111,0.083333,0.0,0.0,0.795260,0.111111,0.083333,0.0,0.0,0.759333,0.092593,0.083333,0.0,0.0,0.759442,0.111111,0.083333,0.0,0.0,0.758576,0.111111,0.083333,0.0,0.0,0.692349,0.111111,0.083333,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,0.836057,0.203704,0.083333,0.0,0.0,0.808029,0.203704,0.083333,0.0

In [36]:
test_temp = X_test[X_test['Time'].isin([7, 7.5])]
test_temp.drop('Time', axis = 1, inplace=True)
train_temp = dataset7[dataset7['Time'].isin([7, 7.5])]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
X = train_temp.iloc[:,:-3]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(train_temp.iloc[:,-2])
y_2 = pd.DataFrame(train_temp.iloc[:,-1])

In [38]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

In [51]:
# X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
# X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

k=0
quantiles = [0.1-k, 0.2-k, 0.3-k, 0.4-k, 0.5, 0.6-k, 0.7-k, 0.8-k, 0.9-k]

In [52]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
#def LGBM(q, X, Y, X_test):

def LGBM(q, X_train, X_valid, Y_train, Y_valid, X_test):    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q, max_depth = 25, num_leaves = 40, min_child_samples = 10, \
                          min_child_weight = 0.1, min_split_gain = 0, n_estimators=20000, bagging_fraction=0.8,\
                          learning_rate=0.02, save_binary = True, subsample=0.8, is_unbalance = False, max_drop = 50, max_bin = 256,\
                          drop_rate  = 0.1, boosting_type = "gbdt", device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
    
 #   splits = 3
 #   kfold = KFold(n_splits = splits, shuffle = True, random_state = 42)
 #   for n_fold, (trn_idx, val_idx) in enumerate(kfold.split(X)):
 #     X_train, Y_train = X.iloc[trn_idx], Y.iloc[trn_idx]
 #     X_valid, Y_valid = X.iloc[val_idx], Y.iloc[val_idx]
 
#    rnd_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=100, cv=4, scoring='neg_mean_squared_error', random_state=42)
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=2000, verbose = 5000)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [53]:
# Target 예측

#def train_data(X_train, Y_train, X_test):

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):


    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, X_valid, Y_train, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [ ]:
tick = time.time()
k=0
quantiles = [0.1-k, 0.2-k, 0.3-k, 0.4-k, 0.5, 0.6-k, 0.7-k, 0.8-k, 0.9-k]

for i in range(5, 20):
  print('##################################################',i,'/19 ##################################################################')
  test_temp = X_test[X_test['Time'].isin([i, i+0.5])]
  test_temp.drop('Time', axis = 1, inplace=True)
  train_temp = dataset7[dataset7['Time'].isin([i, i+0.5])]

  X = train_temp.iloc[:,:-3]
  #X = encoding(X)
  #X.drop('season', axis =1, inplace=True)

  y_1 = pd.DataFrame(train_temp.iloc[:,-2])
  y_2 = pd.DataFrame(train_temp.iloc[:,-1])

  X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
  X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

  models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, test_temp)
  models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, test_temp)

  submission.loc[(submission.id.str.contains("Day7")) & (submission.id.str.contains("_%dh" %i)), "q_0.1":] = results_1.sort_index().values
  submission.loc[(submission.id.str.contains("Day8")) & (submission.id.str.contains("_%dh" %i)), "q_0.1":] = results_2.sort_index().values
  

tock = time.time()

################################################## 5 /19 ##################################################################
0.1


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Training until validation scores don't improve for 2000 rounds
Early stopping, best iteration is:
[235]	valid_0's quantile: 0.110675
0.2
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
Training until validation scores don't improve for 2000 rounds
Early stopping, best iteration is:
[273]	valid_0's quantile: 0.159061
0.3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
Training until validation scores don't improve for 2000 rounds
Early stopping, best iteration is:
[272]	valid_0's quantile: 0.169301
0.4
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
Training until validation scores don't improve for 2000 rounds
Early stopping, best iteration is:
[334]	valid_0's quantile: 0.177967
0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: baggi

In [ ]:
# 출력
print('소요시간 : ', (tock - tick)/60, '분')
# submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
# submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [ ]:
from scipy.optimize import curve_fit
import pylab
import math

In [ ]:
# columns_IDA는 상하로 맞춰줌
# rows_IDA는 좌우로 맞춰줌
# 상하 > 좌우 & 좌우 > 상하를 비교한 결과 좌우로 먼저 맞춘 다음에 상하를 맞추는 것이 쬐------끔 좋은 결과를 보임.

# 그래서 rows_IDA 먼저 하고 columns_IDA 추천


def columns_IDA(dataset):
  dataset2 = pd.DataFrame()
  for j in range(0,9):
    new_val = []
    for m in range(1, int(raw_data.shape[0]/48)+1):
      temp = dataset.iloc[48*(m-1):48*m,j].values

      strt = 0
      endd = 0
      k = 0

      for i in range(len(temp)):
        if temp[i] > k and k ==0 and i < 24:
          strt = i
        elif temp[i] < k and temp[i] == 0 and i > 24:
          endd = i
        k = temp[i]
      if strt != 0:
        bf = temp[:strt+1]
        y = temp[strt+1:endd-1]
        af = temp[endd-1:]
        # bf = temp[:strt]
        # y = temp[strt:endd]
        # af = temp[endd:]

        x = np.array([i for i in range(len(y))])
        fit_t = np.polyfit(x,y,5)

        for i in x:
          fit1 = fit_t[0]*x**5 + fit_t[1]*x**4 + fit_t[2]*x**3 + fit_t[3]*x**2 + fit_t[4]*x + fit_t[5]
      # fit1 = fit_t[0]*x**4 + fit_t[1]*x**3 + fit_t[2]*x**2 + fit_t[3]*x + fit_t[4]
      # fit1 = fit_t[0]*x**2 + fit_t[1]*x + fit_t[2]

        revised = np.concatenate((bf, fit1, af), axis=0)
      elif strt == 0:
        revised = temp
      new_val.extend(list(revised))
    k = pd.DataFrame(new_val)
    dataset2 = pd.concat([dataset2, k], axis = 1)
  return dataset2

def f(x, a,b,c, d):
  y = 1/3 * b * x**3 - a*b *x**2 + (a**2*b + b*c**2)*x + d
  # y = a*(x-12*a) * x **2 + c
  # y = a*x**2 + b
  return y

def rows_IDA(dataset):
  revised = pd.DataFrame()
  for i in range(dataset.shape[0]):
    y = dataset.iloc[i,:].values
    y = np.array(list(y))
    x = np.array([i for i in range(len(y))])

    if list(y).count(0) <= 5 :
      popt, pcov = curve_fit(f, x, y, maxfev = 999990000)
      yfit = f(x, *popt)
      revised = pd.concat([revised, pd.DataFrame(yfit).transpose()], axis = 0)
    else:
      revised = pd.concat([revised, pd.DataFrame(y).transpose()], axis = 0)
    if i % 1000 == 0:
      print(i, '/7776')
  return revised

In [ ]:
test = rows_IDA(submission3)
dataset2 = not_minus(test)

In [ ]:
dataset3 = pd.DataFrame(dataset2.values, columns = submission3.columns, index = submission3.index)

In [ ]:
dataset3.dropna()

In [ ]:
dataset4 = columns_IDA(dataset3)
dataset5 = pd.DataFrame(dataset4.values, columns = submission3.columns, index = submission3.index)
submission3 = not_minus(dataset5)

In [ ]:
submission3.to_csv('test.csv')
!cp test.csv "drive/My Drive/"